# Rekognition による顔検出・顔識別

## 必要なパッケージのインポート

In [ ]:
import io
import requests
import time
import boto3
import numpy as np
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt

## クライアントの作成

In [ ]:
client = boto3.client('rekognition')

## 顔コレクションの作成

In [ ]:
COLLECTION_ID = 'test' + str(time.time())
COLLECTION_ID

In [ ]:
client.create_collection(CollectionId=COLLECTION_ID)

## 顔コレクションへの登録

In [ ]:
face_list = []

In [ ]:
img_a = Image.open('imgs/face_a.jpg')
img_a

In [ ]:
img_bytes_io = io.BytesIO()
img_a.save(img_bytes_io, format="jpeg")
img_bytes = img_bytes_io.getvalue()

response = client.index_faces(
    CollectionId=COLLECTION_ID,
    Image={
        'Bytes': img_bytes
    },
    DetectionAttributes=["ALL"]
)

response

In [ ]:
def show_faces(img, records):
    img_width, img_height = img.size
    draw = ImageDraw.Draw(img)

    for record in records:
        face = record['Face']
        box = face['BoundingBox']
        img_face = img.crop(
            (
                box['Left'] * img_width, box['Top'] * img_height, # 左上座標
                (box['Left'] + box['Width']) * img_width, (box['Top'] + box['Height']) * img_height # 右下座標
            )
        )
        plt.imshow(np.asarray(img_face))
        plt.show()
        print(face['FaceId'], ' : ' + str(round(face['Confidence'], 1)) + ' %' )
        
        detail = record['FaceDetail']
        print('Gender : ' + detail['Gender']['Value'], ' : ' + str(round(detail['Gender']['Confidence'], 1)) + ' %' )
        print('EyesOpen : ' + str(detail['EyesOpen']['Value']), ' : ' + str(round(detail['EyesOpen']['Confidence'], 1)) + ' %' )
        print('Smile : ' + str(detail['Smile']['Value']), ' : ' + str(round(detail['Smile']['Confidence'], 1)) + ' %' )

        for emotion in detail['Emotions']:
            print(emotion['Type'], ' : ' + str(round(emotion['Confidence'], 1)) + ' %' )
        
        face_list.append({
            'FaceId': face['FaceId'],
            'Image': img_face
        })

In [ ]:
show_faces(img_a.copy(), response['FaceRecords'])

In [ ]:
img_b = Image.open('imgs/face_b.jpg')
img_b

In [ ]:
img_bytes_io = io.BytesIO()
img_b.save(img_bytes_io, format="jpeg")
img_bytes = img_bytes_io.getvalue()

response = client.index_faces(
    CollectionId=COLLECTION_ID,
    Image={
        'Bytes': img_bytes
    },
    DetectionAttributes=["ALL"]
)
show_faces(img_b.copy(), response['FaceRecords'])

In [ ]:
img_c = Image.open('imgs/face_c.jpg')
img_c

In [ ]:
img_bytes_io = io.BytesIO()
img_c.save(img_bytes_io, format="jpeg")
img_bytes = img_bytes_io.getvalue()

response = client.index_faces(
    CollectionId=COLLECTION_ID,
    Image={
        'Bytes': img_bytes
    },
    DetectionAttributes=["ALL"]
)
show_faces(img_c.copy(), response['FaceRecords'])

## 同一人物の検索

In [ ]:
for face in face_list:
        print('対象人物 : ' + face['FaceId'])
        plt.imshow(np.asarray(face['Image']))
        plt.show()
        print('同一人物')
        response = client.search_faces(
            CollectionId=COLLECTION_ID,
            FaceId=face['FaceId']
        )
        for matche in response['FaceMatches']:
            print(matche['Face']['FaceId'], ' : ' + str(round(matche['Similarity'], 1)) + ' %' )
            for sim_face in face_list:
                if sim_face['FaceId'] == matche['Face']['FaceId']:
                    plt.imshow(np.asarray(sim_face['Image']))
                    plt.show()
                    break
        print('')
        print('')
        print('')

## 顔コレクションの削除

In [ ]:
client.delete_collection(CollectionId=COLLECTION_ID)